### Predicting the CDR - The Clinical Dementia Rating. Higher the number, higher the chances of dementia

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("OASIS4_data_CDR.csv")

In [3]:
# Import the necessary module
from ydata_profiling import ProfileReport

# Create a ProfileReport object
report = ProfileReport(df)

# Generate the data profiling report
report.to_file('oasis_4_report.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
#print number of rows and columns
print(df.shape)

(1158, 11)


In [13]:
# impute the missing values of columns 3 onwards with the mean of the respective columns
df.fillna(df.mean(), inplace=True)

C:\Users\Ananya\AppData\Local\Temp\ipykernel_19212\906269674.py:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.fillna(df.mean(), inplace=True)


In [16]:
df.isnull().sum()

oasis_id      0
cdr_id        0
visit_days    0
memory        0
orient        0
judgement     0
community     0
homehobb      0
perscare      0
sumbox        0
cdr           0
dtype: int64

In [20]:
# check all the columns in df and convert all the int columns to float
for col in df.columns:
    if df[col].dtype == 'int64':
        df[col] = df[col].astype('float64')

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1158 entries, 0 to 1157
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   oasis_id    1158 non-null   object 
 1   cdr_id      1158 non-null   object 
 2   visit_days  1158 non-null   float64
 3   memory      1158 non-null   float64
 4   orient      1158 non-null   float64
 5   judgement   1158 non-null   float64
 6   community   1158 non-null   float64
 7   homehobb    1158 non-null   float64
 8   perscare    1158 non-null   float64
 9   sumbox      1158 non-null   float64
 10  cdr         1158 non-null   float64
dtypes: float64(9), object(2)
memory usage: 99.6+ KB


In [24]:
df = df.drop(['oasis_id', 'cdr_id'], axis=1)

#### Splitting into Train and Test

In [25]:
# split into x and y where y is 'Group' 
x = df.drop(columns=['cdr'])
y = df['cdr']